In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(grf)
# install.packages("grf")

In [3]:
%%R -o predict_test_z -o test_z
i=1
train_file <- paste("csv_files/train", as.character(i-1),".csv", sep="")
train_x <- read.csv(train_file)
testing_file <- paste("csv_files/test_data", as.character(i-1),".csv", sep="")
testing_x <- read.csv(testing_file)
train_labels <- paste("csv_files/train_label", as.character(i-1),".csv", sep="")
train_labels <- read.csv(train_labels)
test_labels <- paste("csv_files/test_label", as.character(i-1),".csv", sep="")
test_labels <- read.csv(test_labels)
train_y = train_labels$X1
train_z = train_labels$X0
test_y = test_labels$X1
test_z = test_labels$X0
tau.forest <- causal_forest(train_x, train_y , train_z)
ite <- predict(tau.forest,newdata=testing_x)$predictions
actual_ite <- paste("csv_files/ite_test", as.character(i-1),".csv", sep="")  
actual_ite <- read.csv(actual_ite)
pehe<-mean((ite-actual_ite$X0)^2)
args.orthog <- list(X = train_x)
forest.W <- do.call(regression_forest, c(Y = list(train_z), args.orthog))
predict_test_z <- predict(forest.W,newdata=testing_x)[, c(1)]

In [4]:
%%R
#calculate rpehe
library(grf)

compute <- function(i,rpehe) {
library('PRROC')
train_file <- paste("csv_files/1000_train", as.character(i-1),".csv", sep="")
train_x <- read.csv(train_file)
testing_file <- paste("csv_files/1000_test", as.character(i-1),".csv", sep="")
testing_x <- read.csv(testing_file)
train_y_file = paste("csv_files/1000_train_yf", as.character(i-1),".csv", sep="")
train_y <- read.csv(train_y_file)$X0
train_z_file = paste("csv_files/1000_train_t", as.character(i-1),".csv", sep="")
train_z <- read.csv(train_z_file)$X0
tau.forest <- causal_forest(train_x, train_y , train_z)
ite <- predict(tau.forest,newdata=testing_x)$predictions
actual_ite <- paste("csv_files/1000_test_ite", as.character(i-1),".csv", sep="")  
actual_ite <- read.csv(actual_ite)
pehe<-mean((ite-actual_ite$X0)^2)
rmse<-sqrt(pehe)
print(pehe)
print(rmse)
rpehe[i] <- rmse

# args.orthog <- list(X = train_x)
# forest.W <- do.call(regression_forest, c(Y = list(train_z), args.orthog))
# predict_test_z <- predict(forest.W,newdata=testing_x)[, c(1)]
    
# roc<-roc.curve(scores.class0 = test_z, scores.class1 = )
# pr<-pr.curve(scores.class0 = test_z, scores.class1 = predict_test_z)

# print(roc)
# print(pr)
    
return(rpehe)
    }
i <- 1
rpehe <- rep(0, 1000)
while(i<=1000) {
    rpehe = compute(i,rpehe)
    i <- i + 1
}


[1] 0.4221064
[1] 0.6496972
[1] 0.5574087
[1] 0.7465981
[1] 0.4269459
[1] 0.6534109
[1] 1.939195
[1] 1.39255
[1] 5.072717
[1] 2.252269
[1] 0.2680727
[1] 0.5177574
[1] 0.1336483
[1] 0.3655795
[1] 0.7265683
[1] 0.8523897
[1] 266.1306
[1] 16.31351
[1] 43.51151
[1] 6.596325
[1] 2.00106
[1] 1.414588
[1] 7.70234
[1] 2.775309
[1] 570.8482
[1] 23.89243
[1] 0.1869281
[1] 0.4323518
[1] 3.181657
[1] 1.78372
[1] 1.136256
[1] 1.065953
[1] 15.60098
[1] 3.949808
[1] 3.436995
[1] 1.853913
[1] 6.401027
[1] 2.530025
[1] 0.985681
[1] 0.9928147
[1] 51.01802
[1] 7.14269
[1] 0.6959232
[1] 0.8342201
[1] 9.212205
[1] 3.035161
[1] 21.57726
[1] 4.645133
[1] 3.065906
[1] 1.750973
[1] 99.58601
[1] 9.979279
[1] 1.080124
[1] 1.03929
[1] 801.2004
[1] 28.30548
[1] 3.625992
[1] 1.904204
[1] 4.794704
[1] 2.189681
[1] 2.720783
[1] 1.64948
[1] 0.3018047
[1] 0.5493675
[1] 5.904778
[1] 2.429975
[1] 1117.512
[1] 33.42921
[1] 14.36789
[1] 3.7905
[1] 22.07715
[1] 4.698633
[1] 72.2221
[1] 8.498359
[1] 2.112287
[1] 1.453371
[1]

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
print(roc_auc_score(test_z,predict_test_z))
print(average_precision_score(test_z,predict_test_z))

0.6967213114754098
0.37904124471559675


In [6]:
%%R
mean(rpehe)

[1] 4.184675


In [7]:
%%R -o rpehe
rpehe

   [1]  0.64969719  0.74659806  0.65341095  1.39254981  2.25226934  0.51775737
   [7]  0.36557947  0.85238973 16.31350897  6.59632523  1.41458829  2.77530901
  [13] 23.89242996  0.43235178  1.78371999  1.06595293  3.94980792  1.85391343
  [19]  2.53002508  0.99281467  7.14269032  0.83422010  3.03516143  4.64513268
  [25]  1.75097292  9.97927925  1.03929017 28.30548384  1.90420366  2.18968125
  [31]  1.64947973  0.54936755  2.42997494 33.42921420  3.79050019  4.69863257
  [37]  8.49835860  1.45337104  4.62082093  4.08048153  0.72766605  0.63195814
  [43]  1.27265515  1.49126880  3.22153774  2.52407749  3.38950656  1.14394599
  [49]  3.54292807  1.28692602  1.22927826  2.08028369  9.30031477  6.66313407
  [55]  0.75872323  0.17254005  0.29215546  2.57706523  0.77809998 10.87990206
  [61]  0.72866484  1.23144333  1.99738717  1.68784171  4.11478465  0.99607196
  [67]  0.68691892 17.06934076  0.70269733  0.12966492  5.20429708  0.10959460
  [73]  0.84096337  0.30627590  0.14926189  4.426218

In [8]:
from scipy.stats import sem
sem(rpehe)

0.19577717869160582

In [9]:
%%R
#calculate treatment prediction
library(grf)

compute <- function(i,predicted_z) {

train_file <- paste("csv_files/1000_train", as.character(i-1),".csv", sep="")
train_x <- read.csv(train_file)
testing_file <- paste("csv_files/1000_test", as.character(i-1),".csv", sep="")
testing_x <- read.csv(testing_file)
train_y_file = paste("csv_files/1000_train_yf", as.character(i-1),".csv", sep="")
train_y <- read.csv(train_y_file)$X0
train_z_file = paste("csv_files/1000_train_t", as.character(i-1),".csv", sep="")
train_z <- read.csv(train_z_file)$X0
tau.forest <- causal_forest(train_x, train_y , train_z)
ite <- predict(tau.forest,newdata=testing_x)$predictions
actual_ite <- paste("csv_files/1000_test_ite", as.character(i-1),".csv", sep="")  
actual_ite <- read.csv(actual_ite)
pehe<-mean((ite-actual_ite$X0)^2)
rmse<-sqrt(pehe)

args.orthog <- list(X = train_x)
forest.W <- do.call(regression_forest, c(Y = list(train_z), args.orthog))
predict_test_z <- predict(forest.W,newdata=testing_x)[, c(1)]
predicted_z[ ,i]<-predict_test_z
    
return(predicted_z)
    }
i <- 1
predicted_z <- matrix(rep(0), ncol = 1000, nrow=75)
while(i<=1000) {
    predicted_z = compute(i,predicted_z)
    i <- i + 1
}


In [10]:
%%R -o predicted_z
predicted_z

           [,1]      [,2]      [,3]      [,4]      [,5]      [,6]      [,7]
 [1,] 0.1446211 0.1415470 0.2686620 0.1408121 0.1453649 0.1823584 0.1445245
 [2,] 0.2636543 0.2153858 0.1514290 0.1974901 0.1393147 0.2187170 0.1701113
 [3,] 0.1599876 0.2664407 0.2156950 0.2121465 0.1563334 0.2460011 0.1367237
 [4,] 0.1311861 0.1914729 0.2478309 0.1747562 0.1364823 0.2012216 0.1161866
 [5,] 0.1315665 0.1253132 0.1831112 0.2666135 0.1385004 0.1595952 0.1615919
 [6,] 0.1472688 0.2086183 0.2438363 0.1842990 0.2457006 0.1619905 0.1553938
 [7,] 0.2392328 0.1687102 0.1089224 0.1964099 0.1496451 0.1205388 0.1976726
 [8,] 0.1252655 0.1310267 0.1542109 0.1926177 0.2041845 0.2381605 0.2006220
 [9,] 0.2139382 0.1514329 0.2404205 0.1438591 0.1361398 0.1887437 0.1298404
[10,] 0.1866568 0.1912368 0.2618420 0.1531729 0.2012955 0.1991280 0.2223196
[11,] 0.2049751 0.2436376 0.2069936 0.1592878 0.2357867 0.1361719 0.2393289
[12,] 0.2550203 0.1779699 0.1583276 0.1916105 0.2036459 0.2625942 0.1670463
[13,] 0.1874

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 0.1869329 0.2381289 0.1918077 0.1862577 0.2319850 0.1704939 0.1824373
 [4,] 0.1546189 0.1795982 0.2149524 0.1556844 0.1892320 0.2241351 0.2215945
 [5,] 0.2384033 0.1280135 0.2499793 0.1302607 0.2667584 0.1352305 0.2060677
 [6,] 0.2858625 0.1451704 0.1310459 0.1362664 0.2448855 0.1650362 0.1633611
 [7,] 0.1975183 0.1218595 0.1629040 0.2252664 0.1946112 0.2410690 0.2191491
 [8,] 0.1993577 0.1595571 0.1742911 0.1710269 0.2433443 0.2167002 0.2590441
 [9,] 0.2343442 0.2509685 0.1456183 0.1166213 0.1809606 0.1809081 0.1081405
[10,] 0.2275525 0.1242513 0.1354687 0.1540468 0.2110673 0.1723376 0.1975808
[11,] 0.1740558 0.1959605 0.1966632 0.1718855 0.1837395 0.1835663 0.2471531
[12,] 0.1191590 0.1775912 0.1494691 0.1412196 0.2331743 0.1279872 0.1288319
[13,] 0.1328670 0.2589424 0.1226566 0.1564045 0.2197439 0.1265408 0.2552892
[14,] 0.2299761 0.2380639 0.1990813 0.1608506 0.2535700 0.1291684 0.1564602
[15,] 0.1693088 0.2454965 0.2422891 0.1393260 0.1828618 0.1608358 0.1627481
[16,] 0.1806902 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 0.1922720
[12,] 0.1446636 0.2297457 0.2173116 0.1721274 0.1332674 0.2097137 0.1958161
[13,] 0.1367433 0.1536397 0.2676482 0.2057212 0.2694938 0.1648318 0.1177065
[14,] 0.1988209 0.1944501 0.1738373 0.2836120 0.2552713 0.1681501 0.1467563
[15,] 0.1576916 0.2187527 0.2202782 0.1541725 0.1566199 0.2434862 0.1849437
[16,] 0.1605826 0.1928317 0.2150412 0.1796960 0.2242223 0.1963802 0.1575814
[17,] 0.1491229 0.1328315 0.1876207 0.1774379 0.1506511 0.2105645 0.2329729
[18,] 0.1426515 0.1354373 0.2535513 0.2517078 0.2159746 0.1632959 0.1476136
[19,] 0.2504787 0.1778754 0.1193808 0.2402422 0.1813252 0.1520371 0.2524572
[20,] 0.1994541 0.1448569 0.1179022 0.1688854 0.1270617 0.1417030 0.2326991
[21,] 0.1251533 0.1709337 0.1527383 0.2384221 0.2111591 0.1407113 0.2917712
[22,] 0.1600309 0.1795935 0.1308512 0.2449693 0.1658302 0.2497193 0.2399622
[23,] 0.2537567 0.2345669 0.1361252 0.1965634 0.2484832 0.2269690 0.1590176
[24,] 0.2448601 0.2301722 0.2162082 0.2086581 0.1754598 0.1840043 0.2154718
[

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import pandas as pd
roc = []
prc = []
for i in range(1000):
    estimated_z = predicted_z[:,i]
    test_z = pd.read_csv('csv_files/1000_test_t'+str(i)+'.csv')
    roc.append(roc_auc_score(test_z,estimated_z))
    prc.append(average_precision_score(test_z,estimated_z))

In [12]:
import numpy as np
print(np.array(roc).mean())
print(np.array(prc).mean())

0.649479166464524
0.30354531419358305
